# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [38]:
import pandas as pd
import requests
from IPython.display import display, Image, Markdown
from bs4 import BeautifulSoup

url = "https://www.imdb.com/list/ls002206094/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Lists to hold extracted data
positions = []
movies = []
years = []
ratings = []
runtimes = []
genres = []
descriptions = []
directors = []
stars = []
votes = []
gross = []
links = []
images = []

components = soup.find_all('div', class_='lister-item mode-detail')

for component in components:
    position = component.find('span', class_='lister-item-index unbold text-primary').text.strip()
    positions.append(position)
    
    movie = component.find('h3').find('a').text.strip()  # More specific to ensure the correct <a> tag is selected
    movies.append(movie)
    
    year = component.find('span', class_='lister-item-year text-muted unbold').text.strip('() ')
    years.append(year)
    
    rating = component.find('span', class_='ipl-rating-star__rating')
    ratings.append(rating.text.strip() if rating else 'N/A') 
    
    runtime = component.find('span', class_='runtime')
    runtimes.append(runtime.text.strip() if runtime else 'N/A')
    
    genre = component.find('span', class_='genre')
    genres.append(genre.text.strip() if genre else 'N/A')
    
    description = component.find_all('p')[1] 
    descriptions.append(description.text.strip() if description else 'N/A')
    
    director_star_text = component.find_all('p')[2].text.strip()
    parts = [part.strip() for part in director_star_text.split('|')]
    
    if parts:
        # Directors
        directors_part = parts[0].replace('Director:', '').replace('Directors:', '').strip()
        directors_names = directors_part.replace('\n', '').strip()
        directors.append(directors_names)
        
        # Stars
        if len(parts) > 1:
            stars_part = parts[1].replace('Stars:', '').strip()
            stars_names = stars_part.replace('\n', '').strip()
            stars.append(stars_names)
        else:
            stars.append('N/A')
    
    nv_elements = component.find_all('span', attrs={'name': 'nv'})
    
    votes.append(nv_elements[0].text if nv_elements else 'N/A')
    
    gross.append(nv_elements[1].text if len(nv_elements) > 1 else 'N/A')

    link = component.find('h3', class_='lister-item-header').find('a', href=True)['href']
    link = "https://www.imdb.com" + link
    links.append(link)
    
    image_element = component.find('img', class_='loadlate')
    if image_element:
        image_url = image_element.get('loadlate')
        images.append(image_url)
    else:
        images.append(None)

df_movies = pd.DataFrame({
    'Position': positions,
    'Movie': movies,
    'Year': years,
    'Rating': ratings,
    'Runtime': runtimes,
    'Genre': genres,
    'Description': descriptions,
    'Director': directors,
    'Stars': stars,
    'Votes': votes,
    'Gross': gross,
    'Links': links,
    'Image': images
})

df_movies['Rating'] = pd.to_numeric(df_movies['Rating'], errors='coerce')

for index, row in df_movies.iterrows():
    #if row['Rating'] >= 9: #### Playing with the code to see just movies that have rating higher than 9.0 removing this one and indenting it will show the whole list
        links = row['Links'] if row['Links'] != 'N/A' else "#"
        
        display(Markdown(f"**Position:** {row['Position']}  \n**Movie:** {row['Movie']}  \n**Year:** {row['Year']}  \n**Rating:** {row['Rating']}  \n**Runtime:** {row['Runtime']}  \n**Genre:** {row['Genre']}  \n**Description:** {row['Description']}  \n**Director:** {row['Director']}  \n**Stars:** {row['Stars']}  \n**Votes:** {row['Votes']}  \n**Gross:** {row['Gross']}  \n**Details:** [Link]({links})"))
    
        if row['Image'] != 'N/A':
            display(Image(url=row['Image']))

**Position:** 1.  
**Movie:** Matrix  
**Year:** 1999  
**Rating:** 8.7  
**Runtime:** 136 min  
**Genre:** Action, Sci-Fi  
**Description:** When a beautiful stranger leads computer hacker Neo to a forbidding underworld, he discovers the shocking truth--the life he knows is the elaborate deception of an evil cyber-intelligence.  
**Director:** Lana Wachowski, Lilly Wachowski  
**Stars:** Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving  
**Votes:** 2,036,639  
**Gross:** $171.48M  
**Details:** [Link](https://www.imdb.com/title/tt0133093/)

**Position:** 2.  
**Movie:** Sieben  
**Year:** 1995  
**Rating:** 8.6  
**Runtime:** 127 min  
**Genre:** Crime, Drama, Mystery  
**Description:** Two detectives, a rookie and a veteran, hunt a serial killer who uses the seven deadly sins as his motives.  
**Director:** David Fincher  
**Stars:** Morgan Freeman, Brad Pitt, Kevin Spacey, Andrew Kevin Walker  
**Votes:** 1,783,439  
**Gross:** $100.13M  
**Details:** [Link](https://www.imdb.com/title/tt0114369/)

**Position:** 3.  
**Movie:** Die Verurteilten  
**Year:** 1994  
**Rating:** 9.3  
**Runtime:** 142 min  
**Genre:** Drama  
**Description:** Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.  
**Director:** Frank Darabont  
**Stars:** Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler  
**Votes:** 2,866,404  
**Gross:** $28.34M  
**Details:** [Link](https://www.imdb.com/title/tt0111161/)

**Position:** 4.  
**Movie:** Die üblichen Verdächtigen  
**Year:** 1995  
**Rating:** 8.5  
**Runtime:** 106 min  
**Genre:** Crime, Drama, Mystery  
**Description:** The sole survivor of a pier shoot-out tells the story of how a notorious criminal influenced the events that began with five criminals meeting in a seemingly random police lineup.  
**Director:** Bryan Singer  
**Stars:** Kevin Spacey, Gabriel Byrne, Chazz Palminteri, Stephen Baldwin  
**Votes:** 1,139,100  
**Gross:** $23.34M  
**Details:** [Link](https://www.imdb.com/title/tt0114814/)

**Position:** 5.  
**Movie:** Pulp Fiction  
**Year:** 1994  
**Rating:** 8.9  
**Runtime:** 154 min  
**Genre:** Crime, Drama  
**Description:** The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.  
**Director:** Quentin Tarantino  
**Stars:** John Travolta, Uma Thurman, Samuel L. Jackson, Bruce Willis  
**Votes:** 2,201,373  
**Gross:** $107.93M  
**Details:** [Link](https://www.imdb.com/title/tt0110912/)

...

**Position:** 96.  
**Movie:** Elephant  
**Year:** 2003  
**Rating:** 7.1  
**Runtime:** 81 min  
**Genre:** Crime, Drama, Thriller  
**Description:** Several ordinary high school students go through their daily routine as two others prepare for something more malevolent.  
**Director:** Gus Van Sant  
**Stars:** Elias McConnell, Alex Frost, Eric Deulen, John Robinson  
**Votes:** 96,784  
**Gross:** $1.23M  
**Details:** [Link](https://www.imdb.com/title/tt0363589/)

**Position:** 97.  
**Movie:** Mystic River  
**Year:** 2003  
**Rating:** 7.9  
**Runtime:** 138 min  
**Genre:** Crime, Drama, Mystery  
**Description:** The tragic murder of a 19-year-old girl reunites three childhood friends still living in Boston--the victim's gangster father, a detective, and the disturbed man they both suspect of killing her.  
**Director:** Clint Eastwood  
**Stars:** Sean Penn, Tim Robbins, Kevin Bacon, Emmy Rossum  
**Votes:** 483,266  
**Gross:** $90.14M  
**Details:** [Link](https://www.imdb.com/title/tt0327056/)

**Position:** 98.  
**Movie:** Der Exorzist  
**Year:** 1973  
**Rating:** 8.1  
**Runtime:** 122 min  
**Genre:** Horror  
**Description:** When a young girl is possessed by a mysterious entity, her mother seeks the help of two Catholic priests to save her life.  
**Director:** William Friedkin  
**Stars:** Ellen Burstyn, Max von Sydow, Linda Blair, Lee J. Cobb  
**Votes:** 451,791  
**Gross:** $232.91M  
**Details:** [Link](https://www.imdb.com/title/tt0070047/)

**Position:** 99.  
**Movie:** Der letzte Kuss  
**Year:** 2006  
**Rating:** 6.4  
**Runtime:** 115 min  
**Genre:** Comedy, Drama, Romance  
**Description:** Michael, at 30, has a great job, still have his four best friends, and has a beautiful girlfriend, but his life seems predictable. Until he meets a college girl. It seems that everybody's having relationship problems.  
**Director:** Tony Goldwyn  
**Stars:** Zach Braff, Jacinda Barrett, Rachel Bilson, Casey Affleck  
**Votes:** 42,213  
**Gross:** $11.60M  
**Details:** [Link](https://www.imdb.com/title/tt0434139/)

**Position:** 100.  
**Movie:** Hundstage  
**Year:** 1975  
**Rating:** 8.0  
**Runtime:** 125 min  
**Genre:** Biography, Crime, Drama  
**Description:** Three amateur bank robbers plan to hold up a bank. A nice simple robbery: Walk in, take the money, and run. Unfortunately, the supposedly uncomplicated heist suddenly becomes a bizarre nightmare as everything that could go wrong does.  
**Director:** Sidney Lumet  
**Stars:** Al Pacino, John Cazale, Penelope Allen, Sully Boyar  
**Votes:** 271,765  
**Gross:** $50.00M  
**Details:** [Link](https://www.imdb.com/title/tt0072890/)

## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here